# 1. Multi-Layer RNN 구현하기

In [8]:
from tensorflow import keras
from keras.datasets import imdb

In [9]:
# 가장 자주 나타나는 단어 300개만 사용
(train_data, train_target), (test_data, test_target) = imdb.load_data(num_words=300)

17473536/17464789 [==============================] - 0s 0us/step


In [10]:
print(train_data.shape, train_target.shape)
print(test_data.shape, test_target.shape)

(25000,) (25000,)
(25000,) (25000,)


In [11]:
print(train_data[:3])

[list([1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 2, 284, 5, 150, 4, 172, 112, 167, 2, 2, 2, 39, 4, 172, 2, 2, 17, 2, 38, 13, 2, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 2, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 2, 12, 8, 2, 8, 106, 5, 4, 2, 2, 16, 2, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 2, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 7, 4, 2, 2, 13, 104, 88, 4, 2, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 2, 26, 2, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32])
 list([1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 2, 8, 118, 2, 14, 2, 20, 13, 119, 2, 189, 102, 5, 207, 110, 2, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 

In [12]:
print(train_target[:20])   # 0(부정), 1(긍정)

[1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1]


In [13]:
from sklearn.model_selection import train_test_split

train_data, val_data, train_target, val_target = train_test_split(train_data, train_target, test_size=0.2, random_state=0)

In [14]:
# 리뷰 길이를 100으로 맞추기
# 100보다 큰 경우 앞부분을 자르고, 100보다 작은 경우 앞부분에 0을 넣음
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_data, maxlen=100)
print(train_seq.shape)   # (샘플 갯수, 토큰 개수(타임스텝 개수))

(20000, 100)


In [15]:
val_seq = pad_sequences(val_data, maxlen=100)
print(val_seq.shape)     # (5000, 100)

(5000, 100)


In [16]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Embedding(300,                 # 어휘 사전 크기 (입력데이터크기)
                           5,                   # 임베딩 벡터 크기
                           input_length=100))   # 타임 스텝 크기(시퀀스 길이)
model.add(layers.SimpleRNN(8, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dropout(0.2)))  # 각 스텝별로 출력되는 중간값에 대한 cost를 구해 gradient를 계산하기
model.add(layers.SimpleRNN(8))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))

In [17]:
model.summary()     # 파라메터수 ( 1,757개 )

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 5)            1500      
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 100, 8)            112       
                                                                 
 time_distributed_1 (TimeDis  (None, 100, 8)           0         
 tributed)                                                       
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 8)                 136       
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                      

In [18]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-simplernn2.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(train_seq, train_target, epochs=100,
                     validation_data=(val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 99s 150ms/step - loss: 0.6819 - accuracy: 0.5495 - val_loss: 0.6113 - val_accuracy: 0.6774
Epoch 2/100
625/625 [==============================] - 93s 149ms/step - loss: 0.5739 - accuracy: 0.7171 - val_loss: 0.5465 - val_accuracy: 0.7260
Epoch 3/100
625/625 [==============================] - 94s 150ms/step - loss: 0.5488 - accuracy: 0.7408 - val_loss: 0.5330 - val_accuracy: 0.7390
Epoch 4/100
625/625 [==============================] - 96s 154ms/step - loss: 0.5352 - accuracy: 0.7430 - val_loss: 0.5308 - val_accuracy: 0.7450
Epoch 5/100
625/625 [==============================] - 92s 148ms/step - loss: 0.5261 - accuracy: 0.7534 - val_loss: 0.5371 - val_accuracy: 0.7394
Epoch 6/100
625/625 [==============================] - 91s 146ms/step - loss: 0.5203 - accuracy: 0.7552 - val_loss: 0.5262 - val_accuracy: 0.7410
Epoch 7/100
625/625 [==============================] - 92s 148ms/step - loss: 0.5120 - accuracy: 0.7603 - val_loss: 0.5311 -

In [19]:
test_seq = pad_sequences(test_data, maxlen=100)
model.evaluate(test_seq[:100], test_target[:100])   # loss ( 0.5314 ), accuracy ( 0.74 )

4/4 [==============================] - 0s 20ms/step - loss: 0.5199 - accuracy: 0.7500


[0.5198907852172852, 0.75]